## Dhruv Patel
## dhruv692000@gmail.com

In [11]:
import requests
import urllib.request 
from urllib.request import urlopen
import time
import datetime as dt
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import math as m

In [3]:
url = "https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=325&Tx_State=0&Tx_District=0&Tx_Market=0&DateFrom=01-Mar-2015&DateTo=01-Apr-2015&Fr_Date=01-Mar-2015&To_Date=01-Apr-2015&Tx_Trend=0&Tx_CommodityHead=Almond(Badam)&Tx_StateHead=--Select--&Tx_DistrictHead=--Select--&Tx_MarketHead=--Select--"

In [4]:
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
type(soup)

bs4.BeautifulSoup

In [12]:
day_diff_15 = dt.timedelta(days = 15)
day_diff_1 = dt.timedelta(days = 1)
#start_date = '01-jan-18'
#end_date = '28-feb-21'
start_date = datetime.strptime('01-jan-18', '%d-%b-%y')
end_date = datetime.strptime('28-feb-21', '%d-%b-%y')
a = start_date
dat_lst=[]
while True:
    str_d = a 
    end_d = a + day_diff_15
    if end_d > end_date:
        dat_lst.append([str_d.strftime('%d-%b-%y'), end_date.strftime('%d-%b-%y')])
        break
    dat_lst.append([str_d.strftime('%d-%b-%y'),end_d.strftime('%d-%b-%y')])
    a = end_d + day_diff_1
    
dat_lst[:5]

[['01-Jan-18', '16-Jan-18'],
 ['17-Jan-18', '01-Feb-18'],
 ['02-Feb-18', '17-Feb-18'],
 ['18-Feb-18', '05-Mar-18'],
 ['06-Mar-18', '21-Mar-18']]

In [14]:
lsdiv = soup.findAll("div", {"class": "commodity"})
commo_value_text_dict = {}
print(lsdiv[:1])

[<div class="commodity">
<label>Price/Arrivals</label>
<select id="ddlArrivalPrice" name="ctl00$ddlArrivalPrice">
<option selected="selected" value="0">Price</option>
<option value="1">Arrival</option>
<option value="2">Both</option>
</select>
</div>]


In [16]:
opt_tags = lsdiv[1].findAll('option')
for i in opt_tags:
    commo_value_text_dict[i['value']] = i.text    
del commo_value_text_dict['0']

commo_value_text_dict

{'137': 'Ajwan',
 '281': 'Alasande Gram',
 '325': 'Almond(Badam)',
 '166': 'Alsandikai',
 '86': 'Amaranthus',
 '130': 'Ambada Seed',
 '355': 'Amla(Nelli Kai)',
 '102': 'Amphophalus',
 '209': 'Antawala',
 '379': 'Anthorium',
 '17': 'Apple',
 '326': 'Apricot(Jardalu/Khumani)',
 '140': 'Arecanut(Betelnut/Supari)',
 '49': 'Arhar (Tur/Red Gram)(Whole)',
 '260': 'Arhar Dal(Tur Dal)',
 '83': 'Ashgourd',
 '232': 'Astera',
 '269': 'Avare Dal',
 '28': 'Bajra(Pearl Millet/Cumbu)',
 '274': 'Balekai',
 '204': 'Bamboo',
 '19': 'Banana',
 '90': 'Banana - Green',
 '29': 'Barley (Jau)',
 '321': 'Bay leaf (Tejpatta)',
 '94': 'Beans',
 '262': 'Beaten Rice',
 '157': 'Beetroot',
 '263': 'Bengal Gram Dal (Chana Dal)',
 '6': 'Bengal Gram(Gram)(Whole)',
 '357': 'Ber(Zizyphus/Borehannu)',
 '143': 'Betal Leaves',
 '41': 'Betelnuts',
 '85': 'Bhindi(Ladies Finger)',
 '113': 'Big Gram',
 '51': 'Binoula',
 '81': 'Bitter gourd',
 '8': 'Black Gram (Urd Beans)(Whole)',
 '264': 'Black Gram Dal (Urd Dal)',
 '38': 'Black

In [6]:
def getdf(comm, ds, de):
    column = []
    url = "https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity={0}&Tx_State=0&Tx_District=0&Tx_Market=0&DateFrom={1}&DateTo={2}&Fr_Date=01-Mar-2015&To_Date=01-Apr-2015&Tx_Trend=0&Tx_CommodityHead=Almond(Badam)&Tx_StateHead=--Select--&Tx_DistrictHead=--Select--&Tx_MarketHead=--Select--".format(comm, ds, de)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    tbl = soup.findAll("table",{"id" : "cphBody_GridPriceData"})
    cols = tbl[0].findAll("tr")
    for ir in cols:
        ii = ir.findAll("th")
        for j in ii:
            if j.text !="":
                column.append(j.text)
    dtfrm = pd.DataFrame(columns=column)
    return dtfrm, column

In [13]:
comm_number_list = sorted(list(commo_value_text_dict.keys()))
for i in comm_number_list:
    if i in already_done:
        continue
    time.sleep(15)
    print('item ', i,': ',commo_value_text_dict[str(i)], '\n')
    main_dfrm, colu = getdf(i, dat_lst[0][0], dat_lst[0][1])
    rowcount = 0
    inner_dfrm = main_dfrm.copy()
    for sd, ed in dat_lst:
        url = "https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity={0}&Tx_State=0&Tx_District=0&Tx_Market=0&DateFrom={1}&DateTo={2}&Fr_Date=01-Mar-2015&To_Date=01-Apr-2015&Tx_Trend=0&Tx_CommodityHead=Almond(Badam)&Tx_StateHead=--Select--&Tx_DistrictHead=--Select--&Tx_MarketHead=--Select--".format(i, sd, ed)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        tbl = soup.findAll("table",{"id" : "cphBody_GridPriceData"})
        
        rows = tbl[0].findAll("tr")
        data = []
        for ir in rows:
            ii = ir.findAll("td")
            for j in ii:
                if j.text != "":
                    data.append(j.text)
        data2 =[]
        for ida in data:
            ida = ida.strip()
            ida = ida.strip('\r')
            ida = ida.strip('\n')
            ida = ida.strip('\t')
            if ida != '':
                data2.append(ida)
    
        ind=1
        strd = ''
        
        for iet in range(len(data2)):
            inner_dfrm.loc[m.floor(iet/10), colu[m.floor(iet%10)]] = data2[iet]
        main_dfrm = main_dfrm.append(inner_dfrm, ignore_index = True).copy()
        #print(main_dfrm)
    com_nam = str(commo_value_text_dict[str(i)])
    com_nam = com_nam.replace('/', ' ')    
    main_dfrm.to_csv('D:/Folder/main/!!Nirma/Side_Projects/Data analysis/Auto generated csv/'+com_nam+'.csv')    
    already_done.append(i)
    

item  163 :  Duster Beans 

item  164 :  Capsicum 

item  165 :  Green Avare (W) 

item  166 :  Alsandikai 

item  167 :  Chow Chow 

item  168 :  Drumstick 

item  169 :  Chapparad Avare 

item  17 :  Apple 

item  170 :  Thogrikai 

item  171 :  Leafy Vegetable 

item  172 :  Mango (Raw-Ripe) 

item  173 :  Sweet Pumpkin 

item  174 :  Peas Wet 

item  175 :  Jamamkhan 

item  176 :  Seemebadnekai 

item  177 :  Knool Khol 

item  178 :  Suvarna Gadde 

item  18 :  Orange 

item  180 :  Lime 

item  181 :  Marasebu 

item  182 :  Jack Fruit 

item  183 :  Siddota 

item  184 :  Jamun(Narale Hannu) 

item  185 :  Guava 

item  187 :  Karbuja(Musk Melon) 

item  188 :  Chakotha 

item  189 :  Borehannu 

item  19 :  Banana 

item  190 :  Pomegranate 

item  2 :  Paddy(Dhan)(Common) 

item  20 :  Mango 

item  200 :  Tender Coconut 

item  201 :  Seetafal 

item  203 :  Wood 

item  204 :  Bamboo 

item  205 :  Cane 

item  206 :  Firewood 

item  207 :  Soapnut(Antawala/Retha) 

item  